In [6]:
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
from experiments.Models import BertBaseline


In [7]:
model = BertBaseline()
model.load_model_weights('../models/bert_evidence_baseline_weights.h5')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
test_dataset = pd.read_csv('../data/emrqa_evidence_test.csv')
tokenized_test = tokenizer(test_dataset['question'].to_list(),
                           test_dataset['evidence'].to_list(),
                           padding='max_length',
                           truncation=True,
                           max_length=512,
                           return_tensors='tf')
test_x = (tokenized_test['input_ids'], tokenized_test['attention_mask'])
test_y = np.stack((np.eye(512)[test_dataset['start_token']], np.eye(512)[test_dataset['end_token']]), axis=1)
results = model.model.predict(test_x, batch_size=32)

2024-02-26 16:24:14.177277: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-26 16:24:14.206651: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-26 16:24:14.206827: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, None,                                        

In [34]:
starts = []
ends = []
for i in results:
    i = i.T
    starts.append(np.argmax(i[0]))
    ends.append(np.argmax(i[1]))
predictions = pd.DataFrame({"start": starts, "end": ends})
predictions

,start,end
0,13,18
1,27,29
2,19,29
3,27,31
4,42,43
...,...,...
6701,11,16
6702,18,22
6703,28,28
6704,16,17


In [40]:
starts_truth = []
ends_truth = []
for i in test_y:
    starts_truth.append(np.argmax(i[0]))
    ends_truth.append(np.argmax(i[1]))
truth = pd.DataFrame({"start": starts_truth, "end": ends_truth})
truth

,start,end
0,13,18
1,27,29
2,19,29
3,27,31
4,42,43
...,...,...
6701,11,16
6702,18,22
6703,28,28
6704,16,17


In [53]:
incorrect = 0
for ind, i in enumerate(truth):
    t = truth.iloc[ind].to_numpy()
    p = predictions.iloc[ind].to_numpy()
    if not (sum(t == p) == 2):
        incorrect += 1
print(incorrect)

0


In [50]:
sum(t == p)

2